<a href="https://colab.research.google.com/github/stereofantasy/Twini-Golf/blob/main/INT3405E_Group_Project_BaseModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import cv2
import os
from nltk import wordpunct_tokenize
import re

In [ ]:
!gdown 1hUqu1mbFeTEfBvl-7fc56fHFfCSzIktD

Downloading...
From: https://drive.google.com/uc?id=1hUqu1mbFeTEfBvl-7fc56fHFfCSzIktD
To: /content/ml1m.zip
100% 105M/105M [00:00<00:00, 170MB/s] 


In [ ]:
!unzip -qq ml1m.zip -d ml1m

Archive:  ml1m.zip
   creating: ml1m/content/dataset/
  inflating: ml1m/content/dataset/movies_test.dat  
  inflating: ml1m/content/dataset/users.dat  
  inflating: ml1m/content/dataset/genres.txt  
  inflating: ml1m/content/dataset/ratings.dat  
  inflating: ml1m/content/dataset/movies_train.dat  
   creating: ml1m/content/dataset/ml1m-images/
  inflating: ml1m/content/dataset/ml1m-images/2816.jpg  
  inflating: ml1m/content/dataset/ml1m-images/1043.jpg  
  inflating: ml1m/content/dataset/ml1m-images/941.jpg  
  inflating: ml1m/content/dataset/ml1m-images/2.jpg  
  inflating: ml1m/content/dataset/ml1m-images/160.jpg  
  inflating: ml1m/content/dataset/ml1m-images/1201.jpg  
  inflating: ml1m/content/dataset/ml1m-images/276.jpg  
  inflating: ml1m/content/dataset/ml1m-images/2119.jpg  
  inflating: ml1m/content/dataset/ml1m-images/599.jpg  
  inflating: ml1m/content/dataset/ml1m-images/2829.jpg  
  inflating: ml1m/content/dataset/ml1m-images/3628.jpg  
  inflating: ml1m/content/dataset

In [ ]:
# You need extract file ml1m.zip to folder ml1m before run code

users = pandas.read_csv('ml1m/content/dataset/users.dat', sep='::',
                        engine='python',
                        names=['userid', 'gender', 'age', 'occupation', 'zip']).set_index('userid')
ratings = pandas.read_csv('ml1m/content/dataset/ratings.dat', engine='python',
                          sep='::', names=['userid', 'movieid', 'rating', 'timestamp'])
movies_train = pandas.read_csv('ml1m/content/dataset/movies_train.dat', engine='python',
                         sep='::', names=['movieid', 'title', 'genre'], encoding='latin-1', index_col=False).set_index('movieid')
movies_test = pandas.read_csv('ml1m/content/dataset/movies_test.dat', engine='python',
                         sep='::', names=['movieid', 'title', 'genre'], encoding='latin-1', index_col=False).set_index('movieid')
movies_train['genre'] = movies_train.genre.str.split('|')
movies_test['genre'] = movies_test.genre.str.split('|')

users.age = users.age.astype('category')
users.gender = users.gender.astype('category')
users.occupation = users.occupation.astype('category')
ratings.movieid = ratings.movieid.astype('category')
ratings.userid = ratings.userid.astype('category')


In [ ]:
movies_train

,title,genre
movieid,,
1650,Washington Square (1997),[Drama]
185,"Net, The (1995)","[Sci-Fi, Thriller]"
1377,Batman Returns (1992),"[Action, Adventure, Comedy, Crime]"
3204,"Boys from Brazil, The (1978)",[Thriller]
1901,Dear Jesse (1997),[Documentary]
...,...,...
2539,Analyze This (1999),[Comedy]
3038,"Face in the Crowd, A (1957)",[Drama]
1832,Heaven's Burning (1997),"[Action, Drama]"


## Dataset + Dataloader

In [ ]:
# movies_train = pandas.read_csv('ml1m/dataset/movies_train.dat', engine='python',
#                          sep='::', names=['movieid', 'title', 'genre'], encoding='latin-1', index_col=False).set_index('movieid')

folder_img_path = 'ml1m/content/dataset/ml1m-images'
movies_train['id'] = movies_train.index
movies_train.reset_index(inplace=True)
movies_train['img_path'] = movies_train.apply(lambda row: os.path.join(folder_img_path, f'{row.id}.jpg'), axis = 1)
movies_train

,movieid,title,genre,id,img_path
0,1650,Washington Square (1997),[Drama],1650,ml1m/content/dataset/ml1m-images/1650.jpg
1,185,"Net, The (1995)","[Sci-Fi, Thriller]",185,ml1m/content/dataset/ml1m-images/185.jpg
2,1377,Batman Returns (1992),"[Action, Adventure, Comedy, Crime]",1377,ml1m/content/dataset/ml1m-images/1377.jpg
3,3204,"Boys from Brazil, The (1978)",[Thriller],3204,ml1m/content/dataset/ml1m-images/3204.jpg
4,1901,Dear Jesse (1997),[Documentary],1901,ml1m/content/dataset/ml1m-images/1901.jpg
...,...,...,...,...,...
3101,2539,Analyze This (1999),[Comedy],2539,ml1m/content/dataset/ml1m-images/2539.jpg
3102,3038,"Face in the Crowd, A (1957)",[Drama],3038,ml1m/content/dataset/ml1m-images/3038.jpg
3103,1832,Heaven's Burning (1997),"[Action, Drama]",1832,ml1m/content/dataset/ml1m-images/1832.jpg
3104,657,Yankee Zulu (1994),"[Comedy, Drama]",657,ml1m/content/dataset/ml1m-images/657.jpg


In [ ]:
folder_img_path = 'ml1m/content/dataset/ml1m-images'
movies_test['id'] = movies_test.index
movies_test.reset_index(inplace=True)
movies_test['img_path'] = movies_test.apply(lambda row: os.path.join(folder_img_path, f'{row.id}.jpg'), axis = 1)
movies_test

,movieid,title,genre,id,img_path
0,3397,"Great Muppet Caper, The (1981)","[Children's, Comedy]",3397,ml1m/content/dataset/ml1m-images/3397.jpg
1,2067,Doctor Zhivago (1965),"[Drama, Romance, War]",2067,ml1m/content/dataset/ml1m-images/2067.jpg
2,2651,Frankenstein Meets the Wolf Man (1943),[Horror],2651,ml1m/content/dataset/ml1m-images/2651.jpg
3,2989,For Your Eyes Only (1981),[Action],2989,ml1m/content/dataset/ml1m-images/2989.jpg
4,3415,"Mirror, The (Zerkalo) (1975)",[Drama],3415,ml1m/content/dataset/ml1m-images/3415.jpg
...,...,...,...,...,...
772,2309,"Inheritors, The (Die Siebtelbauern) (1998)",[Drama],2309,ml1m/content/dataset/ml1m-images/2309.jpg
773,2421,"Karate Kid, Part II, The (1986)","[Action, Adventure, Drama]",2421,ml1m/content/dataset/ml1m-images/2421.jpg
774,3255,"League of Their Own, A (1992)","[Comedy, Drama]",3255,ml1m/content/dataset/ml1m-images/3255.jpg
775,974,Algiers (1938),"[Drama, Romance]",974,ml1m/content/dataset/ml1m-images/974.jpg


In [ ]:
def tokenize(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    tokens = wordpunct_tokenize(text)
    tokens = tokens[:-1] # remove last token because it is the year which maybe is not useful
    return tokens

def create_vocab():
    df = movies_train.copy()
    arr_title = df['title'].tolist()
    vocab = set()
    for title in arr_title:
        tokens = tokenize(title)
        vocab.update(tokens)
    vocab = list(vocab)
    pad_token = '<PAD>'
    unk_token = '<UNK>'
    vocab.append(pad_token)
    vocab.append(unk_token)
    return vocab

In [ ]:
class MLDataset(Dataset):
    def __init__(self, is_train=True):
        if is_train:
            self.data =  movies_train
        else:
            self.data = movies_test
        self.data['title_tokens'] = [tokenize(x) for x in self.data.title]

        # create vocab
        vocab = create_vocab()
        pad_token = '<PAD>'
        unk_token = '<UNK>'
        self.token2idx = {token: idx for idx, token in enumerate(vocab)}

        # Create a binary vector for each word in each sentence
        MAX_LENGTH = 7
        vectors = []
        for title_tokens in self.data.title_tokens.tolist():
            if len(title_tokens) < MAX_LENGTH:
                num_pad = MAX_LENGTH - len(title_tokens)
                for idx in range(num_pad):
                    title_tokens.append(pad_token)
            else:
                title_tokens = title_tokens[:MAX_LENGTH]
            title_vectors = []
            for word in title_tokens:
                binary_vector = np.zeros(len(vocab))
                if word in vocab:
                    binary_vector[self.token2idx[word]] = 1
                else:
                    binary_vector[self.token2idx[unk_token]] = 1
                title_vectors.append(binary_vector)

            vectors.append(np.array(title_vectors))
        self.data['vectors'] = vectors

        # label genre
        with open('ml1m/content/dataset/genres.txt', 'r') as f:
            genre_all = f.readlines()
            genre_all = [x.replace('\n','') for x in genre_all]
        self.genre2idx = {genre:idx for idx, genre in enumerate(genre_all)}

    def __getitem__(self, index):
        title = self.data.iloc[index].title
        img_path = self.data.iloc[index].img_path
        genre = self.data.iloc[index].genre

        # preprocess text
        title_vector = self.data.iloc[index].vectors
        title_tensor = torch.from_numpy(title_vector).float()

        # preprocess img
        if os.path.exists(img_path):
            img = cv2.imread(img_path)
        else:
            img = np.random.rand(256,256,3)
        img = cv2.resize(img, (256,256))
        img_tensor = torch.from_numpy(img.transpose(2,0,1)).float()

        # preprocess label
        genre_vector = np.zeros(len(self.genre2idx))

        for g in genre:
            genre_vector[self.genre2idx[g]] = 1
        genre_tensor = torch.from_numpy(genre_vector).float()

        return title_tensor, img_tensor, genre_tensor

    def __len__(self):
        return len(self.data)

In [ ]:
train_set = MLDataset(is_train=True)
test_set = MLDataset(is_train=False)

BATCH_SIZE = 8
train_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_set, batch_size=BATCH_SIZE)

In [ ]:
for title_tensor, img_tensor, genre_tensor in train_dataloader:
    print(title_tensor.shape, img_tensor.shape, genre_tensor.shape)
    break
for title_tensor, img_tensor, genre_tensor in test_dataloader:
    print(title_tensor.shape, img_tensor.shape, genre_tensor.shape)
    break

torch.Size([8, 7, 3899]) torch.Size([8, 3, 256, 256]) torch.Size([8, 18])
torch.Size([8, 7, 3899]) torch.Size([8, 3, 256, 256]) torch.Size([8, 18])


## Model

In [ ]:
with open('ml1m/content/dataset/genres.txt', 'r') as f:
    genre_all = f.readlines()
num_classes = len(genre_all)
num_classes

18

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes, len_vocab, embedding_dimension=3898, hidden_size=64):
        super(BaseModel, self).__init__()
        self.hidden_size = hidden_size

        # img
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.maxpool = nn.MaxPool2d(kernel_size=4, stride=4)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 64 * 64, hidden_size)


        # text
        self.fc2 = nn.Linear(7*len_vocab, hidden_size)

        self.fc3 = nn.Linear(hidden_size*2, num_classes)

    def forward(self, text_tens, img_tens):
        text_feat = self.fc2(self.flatten(text_tens))

        img_feat = self.conv1(img_tensor)
        img_feat = self.maxpool(img_feat)
        img_feat = self.conv2(img_feat)
        img_feat = self.fc1(self.flatten(img_feat))

        out = self.fc3(torch.concat([text_feat, img_feat], dim=1))
        return out

In [ ]:
vocab = create_vocab()
model = BaseModel(num_classes, len(vocab))

out = model(title_tensor, img_tensor)
out.shape

torch.Size([8, 18])

## Loss

In [ ]:
from torch import optim
criterion = nn.CrossEntropyLoss()

learning_rate = 1e-3
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=learning_rate,
)

## Training

In [ ]:
device = torch.device('cuda:0')
NUM_EP = 10
model.to(device)
for ep in range(NUM_EP):
    print('ep:', ep)
    for idx, (title_tensor, img_tensor, genre_tensor) in enumerate(train_dataloader):
        title_tensor = title_tensor.to(device)
        img_tensor = img_tensor.to(device)
        genre_tensor = genre_tensor.to(device)

        out = model(title_tensor, img_tensor)

        loss = criterion(out, genre_tensor)

        loss.backward()
        optimizer.zero_grad()
        optimizer.step()


ep: 0
ep: 1
ep: 2
ep: 3
ep: 4
ep: 5
ep: 6
ep: 7
ep: 8
ep: 9


In [ ]:
!pip install -q torchmetrics
from torchmetrics.classification import MultilabelF1Score

In [ ]:
N, C = genre_tensor.shape

auroc_all = 0
f1_all = 0
f1 = MultilabelF1Score(num_labels=C, threshold=0.5, average='macro')
f1 = f1.to(device)
for title_tensor, img_tensor, genre_tensor in test_dataloader:
    title_tensor = title_tensor.to(device)
    img_tensor = img_tensor.to(device)
    genre_tensor = genre_tensor.to(device)

    out = model(title_tensor, img_tensor)
    f1_batch = f1(out, genre_tensor)
    f1_all += f1_batch

print('F1: ', f1_all/len(test_dataloader))


F1:  tensor(0.0942, device='cuda:0')
